<a href="https://colab.research.google.com/github/raulbenitez/recommenders/blob/main/load_handle_data_surprise_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install suprise libraries in google colab:

In [3]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 5.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618255 sha256=e725045beffa24e869f65152d0350b971eb0ff3cf0e6343ae66f4db044c7939c
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


Load Jesper data from built-in surprise dataset

In [4]:
from surprise import Dataset

data = Dataset.load_builtin('jester')

Dataset jester could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://eigentaste.berkeley.edu/dataset/archive/jester_dataset_2.zip...
Done! Dataset jester has been saved to /root/.surprise_data/jester


Convert data to a Pandas dataframe

In [5]:
import pandas as pd
df = pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id', 'item_id', 'rating', 'timestamp'])

Perform basic manipulation and inspection of data:

List 10 first entries of the dataset:

In [6]:
df.head(10) 

,user_id,item_id,rating,timestamp
0,1,5,0.219,None
1,1,7,-9.281,None
2,1,8,-9.281,None
3,1,13,-6.781,None
4,1,15,0.875,None
5,1,16,-9.656,None
6,1,17,-9.031,None
7,1,18,-7.469,None
8,1,19,-8.719,None
9,1,20,-9.156,None


How many entries are there in the dataset?

In [7]:
df.shape[0] # 0: number of rows, 1: number of columns/variables

1761439

How many users and jokes are there in the database?

In [12]:
print('Number of users:')
df["user_id"].nunique() # number of users

Number of users:


59132

In [13]:
print('Number of items:')
df["item_id"].nunique() # number of items

Number of items:


140

What is the rating of the user with user_id = 1 of the joke with item_id = 5?


In [14]:
df.loc[(df["user_id"]=='1') & (df["item_id"]=='5')]

,user_id,item_id,rating,timestamp
0,1,5,0.219,None


Filter the database keeping only items that have rated at least min_item_ratings users and users that have rated at least min_user_ratings items:

In [15]:
min_item_ratings = 200
filter_items = df['item_id'].value_counts() > min_item_ratings
filter_items = filter_items[filter_items].index.tolist()

min_user_ratings = 130
filter_users = df['user_id'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_small = df[(df['item_id'].isin(filter_items)) & (df['user_id'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_small.shape))

The original data frame shape:	(1761439, 4)
The new data frame shape:	(1325, 4)


Convert dataframe back to a surprise variable using the reader and load_from_df functions:

https://surprise.readthedocs.io/en/stable/getting_started.html#use-a-custom-dataset

In [17]:
from surprise import Reader 
reader = Reader(rating_scale=(-10, 10))
data_small = Dataset.load_from_df(df_small[['user_id', 'item_id', 'rating']], reader)